In [23]:
import mujoco
import mujoco.viewer as viewer
import numpy as np
import time
import maze

# --------------------
# Maze parameters
# --------------------
cell_size = 0.15
wall_thickness = 0.01
wall_height = 0.1
maze_size = 6 * cell_size / 2

remove_walls = [
    (1, 7), (7, 13), (13, 19), (2, 3), (3, 4), (4, 5), (5, 6), (2, 8),
    (3, 9), (9, 10), (10, 11), (11, 12), (10, 16), (16, 22), (22, 28),
    (12, 18), (18, 24), (17, 18), (17, 23), (23, 29), (29, 30), (30, 36),
    (35, 36), (34, 35), (33, 34), (32, 33), (31, 32), (31, 25), (32, 26),
    (26, 27), (19, 20), (20, 14), (14, 15), (15, 21), (21, 27),
]

def should_remove_wall(cell1, cell2):
    return (cell1, cell2) in remove_walls or (cell2, cell1) in remove_walls

# --------------------
# Scaling and offsets
# --------------------
y_shift = 0.8


# --------------------
# Build walls
# --------------------
h_walls = []
for row in range(5):
    for col in range(6):
        cell_above = row * 6 + col + 1
        cell_below = (row + 1) * 6 + col + 1
        if not should_remove_wall(cell_above, cell_below):
            y_pos = maze_size - (row + 1) * cell_size + y_shift
            x_pos = -maze_size + col * cell_size + cell_size / 2
            h_walls.append(f'<body name="h_{row}_{col}" pos="{x_pos} {y_pos} 0">'
                           f'<geom type="box" size="{cell_size/2  + wall_thickness/2} {wall_thickness/2} {wall_height}" '
                           f'rgba="0.7 0.7 0.7 1"/></body>')

v_walls = []
for col in range(5):
    for row in range(6):
        cell_left = row * 6 + col + 1
        cell_right = row * 6 + col + 2
        if not should_remove_wall(cell_left, cell_right):
            x_pos = -maze_size + (col + 1) * cell_size
            y_pos = maze_size - row * cell_size - cell_size / 2 + y_shift
            v_walls.append(f'<body name="v_{row}_{col}" pos="{x_pos} {y_pos} 0">'
                           f'<geom type="box" size="{wall_thickness/2} {cell_size/2 + wall_thickness/2} {wall_height}" '
                           f'rgba="0.7 0.7 0.7 1"/></body>')

# Outer walls
outer_walls = f'''
<body name="wall_top" pos="0 {maze_size + y_shift} 0">
    <geom type="box" size="{maze_size + wall_thickness/2} {wall_thickness/2} {wall_height}" rgba="0.7 0.7 0.7 1"/>
</body>

<body name="wall_bottom" pos="0 {-maze_size + y_shift} 0">
    <geom type="box" size="{maze_size + wall_thickness/2} {wall_thickness/2} {wall_height}" rgba="0.7 0.7 0.7 1"/>
</body>

<body name="wall_left" pos="{-maze_size} {y_shift} 0">
    <geom type="box" size="{wall_thickness/2} {maze_size + wall_thickness/2} {wall_height}" rgba="0.7 0.7 0.7 1"/>
</body>

<body name="wall_right" pos="{maze_size} {y_shift} 0">
    <geom type="box" size="{wall_thickness/2} {maze_size + wall_thickness/2} {wall_height}" rgba="0.7 0.7 0.7 1"/>
</body>
'''

maze_section = outer_walls + chr(10).join(h_walls) + chr(10).join(v_walls)

# --------------------
# Robot & ball parameters
# --------------------
l1, l2, l3 = 0.4, 1.0, 1.0
base_pos_x, base_pos_y, base_pos_z = 0, 0, 0
ball_pos_x = -maze_size + cell_size / 2
ball_pos_y = -maze_size + cell_size / 2 + y_shift
ball_pos_z = 0
ball_radius = 0.03
kp1, kp2, kp3, kp4 = 200, 500, 500, 1
grav = -9.81

# --------------------
# Full Mujoco XML
# --------------------
robot_4dof = f'''
<mujoco model="3dof_arm">
    <compiler angle="radian" coordinate="local" meshdir="assets" autolimits="true"/>
    <option gravity="0 0 {grav}"/>

    <asset>
        <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3" rgb2=".2 .3 .4" width="300" height="300"/>
        <material name="grid" texture="grid" texrepeat="8 8" reflectance=".2"/>
        <mesh name="ee" file="ee1.STL" scale="0.05 0.05 0.05" face="two"/>
    </asset>

    <worldbody>
        <geom size="2 2 0.01" type="plane" material="grid"/>
        <light pos="0 0 1"/>

        <!-- Base and Arm -->
        <body name="base" pos="{base_pos_x} {base_pos_y} {base_pos_z}">
            <body name="joint1" pos="0 0 0">
                <joint name="joint1_hinge" type="hinge" axis="0 0 1"/>
                <geom type="sphere" size="0.05" rgba="1 0 0 1" contype="1" conaffinity="0"/>
                <geom type="cylinder" fromto="0 0 0 0 0 {l1}" size="0.02" rgba="0 1 0 1" contype="1" conaffinity="0"/>
                
                <body name="joint2" pos="0 0 {l1}" gravcomp="1">
                    <joint name="joint2_hinge" type="hinge" axis="0 1 0"/>
                    <geom type="sphere" size="0.05" rgba="1 0 0 1"/>
                    <geom type="cylinder" fromto="0 0 0 0 0 {l2}" size="0.02" rgba="0 1 0 1"/>
                    
                    <body name="joint3" pos="0 0 {l2}" gravcomp="1">
                        <joint name="joint3_hinge" type="hinge" axis="0 1 0"/>
                        <geom type="sphere" size="0.05" rgba="1 0 0 1"/>
                        <geom type="cylinder" fromto="0 0 0 0 0 {l3}" size="0.02" rgba="0 1 0 1"/>

                        <body name="ee" pos="0 0 {l3}" gravcomp="1">
                            <joint name="joint4_hinge" type="hinge" axis="0 1 0" actuatorgravcomp="true"/>
                            <geom type="sphere" size="0.02" rgba="1 1 1 1"/>
                            <geom type="mesh" mesh="ee" />
                        </body>
                    </body>
                </body>
            </body>
        </body>

        <!-- Ball -->
        <body name="ball" pos="{ball_pos_x} {ball_pos_y} {ball_pos_z}">
            <geom type="sphere" size="{ball_radius}" rgba="1 1 0 1"/>
            <joint type="free"/>
        </body>

        <!-- Maze -->
        {maze_section}

    </worldbody>

    <actuator>
        <position joint="joint1_hinge" kp="{kp1}" dampratio="1"/>
        <position joint="joint2_hinge" kp="{kp2}" dampratio="1"/>
        <position joint="joint3_hinge" kp="{kp3}" dampratio="1"/>
        <position joint="joint4_hinge" kp="{kp4}" dampratio="0.5"/>
    </actuator>
</mujoco>
'''

                            # <!-- Fingers -->
                            # <body name="finger1_rot" pos="0 0 0" euler="0 0 0">
                            #     <body name="finger1" pos="0 0 0" euler="-0.7854 0 0">
                            #         <geom type="capsule" fromto="0 0 0 0 0 0.06" size="0.01" rgba="0.8 0.8 0.8 1"/>
                            #     </body>
                            # </body>
                            # <body name="finger2_rot" pos="0 0 0" euler="0 0 2.0944">
                            #     <body name="finger2" pos="0 0 0" euler="-0.7854 0 0">
                            #         <geom type="capsule" fromto="0 0 0 0 0 0.06" size="0.01" rgba="0.8 0.8 0.8 1"/>
                            #     </body>
                            # </body>
                            # <body name="finger3_rot" pos="0 0 0" euler="0 0 4.1888">
                            #     <body name="finger3" pos="0 0 0" euler="-0.7854 0 0">
                            #         <geom type="capsule" fromto="0 0 0 0 0 0.06" size="0.01" rgba="0.8 0.8 0.8 1"/>
                            #     </body>
                            # </body>

In [ ]:
## Simulation with joint positions controllable using GUI
## Use this to manually control the joints and do sanity checks

# Load model to reset robot
model = mujoco.MjModel.from_xml_string(robot_4dof)
data = mujoco.MjData(model)

with viewer.launch(model, data) as v:
    while v.is_running():
        mujoco.mj_step(model, data)
        v.render()

In [ ]:
## IK

def shortest_angle_update(theta_current, theta_dest):
    # Finds closest angle configuration equivalent of goal configuration
    # to current configuration
    #  
    # theta_current: current angle measure
    # theta_dest: desired angle measure
    # returns the closest angle equivalent of theta_dest to theta_current
    diff = theta_dest - theta_current
    diff = (diff + np.pi) % (2*np.pi) - np.pi
    return theta_current + diff

def dist(point1, point2):
    # Finds distance between two points 
    # returns distance between the two points
    return np.linalg.norm(np.array(point2)[:2] - np.array(point1)[:2])

def forward_kinematics(theta1, theta2, theta3, l1, l2, l3):
    # Calculates the ee position given joint angles and arm lengths
    # Returns np.array([x, y, z]) of coordinates
    ##BUG: Returned position is nowhere near actual position
    s1 = np.sin(theta1)
    s2 = np.sin(theta2)
    s3 = np.sin(theta3)
    c1 = np.cos(theta1)
    c2 = np.cos(theta2)
    c3 = np.cos(theta3)
    x = -(l3*(c1*c2*c3 - c1*s2*s3) + c1*c2*l2)
    y = -(l3*(-s1*c2*c3 + s1*s2*s3) - s1*c2*l2)
    z = l3*(s2*c3 + c2*s3) + s2*l2 + l1

    return np.array([x, y, z])

def instructions_to_coordinates(instructions, start=np.array([ball_pos_x, ball_pos_y, ball_pos_z]), step_size=cell_size):
    # instructions: list of "U", "D", "L", "R" strings
    # step_size: distance moved for each instruction
    # returns a list of coordinates
    coordinates = [start]
    z_rise = 0.1 # keep the arm elevated
    for inst in instructions:
        x, y, z = coordinates[-1].copy()
        if inst == "U":
            y += step_size
        elif inst == "D":
            y -= step_size
        elif inst == "R":
            x += step_size
        elif inst == "L":
            x -= step_size
        else:
            raise ValueError(f"Invalid instruction: {inst}")
        coordinates.append(np.array([x, y, z_rise]))
    return coordinates


# Load model
model = mujoco.MjModel.from_xml_string(robot_4dof)
data = mujoco.MjData(model)

# Init parameters
ee_id = model.body('ee').id
base_id = model.body('base').id
ball_id = model.body('ball').id
base_xpos = data.body(base_id).xpos
tol = 0.1 # minimum distance from each step before moving on to the next (tune this)
update_freq = 1/60 # framerate/simulation rate. lower to speed up

# instructions = maze.start_end_to_instruction()
# instructions = ['R', 'R', 'R', 'R', 'R', 'U', 'L', 'U', 'U', 'R', 'U', 'L', 'L', 'L', 'U', 'R', 'R', 'R']
# coordinates = instructions_to_coordinates(instructions, start=start)
# coord_index = 0
t0 = time.time()

with mujoco.viewer.launch_passive(model, data) as v:
    while v.is_running():
        ##TODO: Maze traversal
        # goal = coordinates[coord_index]
        # ball_pos = data.body(ball_id).xpos
        # ee_pos = data.body(ee_id).xpos
        # error = dist(ee_pos, goal)
        # if error < tol:
        #     coord_index += 1
        #     if coord_index >= len(coordinates):
        #         coord_index = len(coordinates) - 1  # Remain at final point

        # draw circle
        t = time.time() - t0
        radius = 0.4
        y_center = 0.3
        x_center = 0
        z_center = 0.7
        omega = 1
        x_goal = radius * np.sin(omega * t) + x_center
        y_goal = y_center
        z_goal = radius * np.cos(omega * t) + z_center
        goal = np.array([x_goal, y_goal, z_goal])

        # Account for base position
        x_offset, y_offset, z_offset = np.subtract(goal, base_xpos)

        # Inverse kinematics
        ##BUG: general path looks decent, but all coordinates have a noticable error
        theta_curr = data.qpos.copy()
        theta1 = np.atan2(-y_offset, -x_offset)
        r = np.sqrt(x_offset**2 + y_offset**2)
        # theta3 = -np.acos((x_offset**2 + y_offset**2 + (z_offset-l1)**2 - l2**2 - l3**2)/(2*l2*l3))
        # theta2 = np.atan2(z_offset-l1, r) - np.atan2(np.sin(theta3)*l3, l2 + np.cos(theta3)*l3)

        theta3 = -np.arccos((x_offset**2 + y_offset**2 + z_offset**2 -l2**2-l3**2)/(2*l2*l3))
        theta2 = np.arcsin(z_offset/np.sqrt(x_offset**2 + y_offset**2 + z_offset**2)) + np.arctan2(l3*np.sin(theta3), l2+l3*np.cos(theta3))
        theta4 = np.pi - (theta2 + theta3)

        # Workaround for snapping between pi and -pi
        theta1_adj = shortest_angle_update(theta_curr[0], theta1)
        theta2_adj = shortest_angle_update(theta_curr[1], theta2)
        theta3_adj = shortest_angle_update(theta_curr[2], theta3)
        theta4_adj = np.pi-(theta2_adj + theta3_adj)

        # Sanity check and debugging
        ee_fk = forward_kinematics(theta1_adj, theta2_adj, theta3_adj, l1, l2, l3)
        print("Goal:\t", goal)
        print("Actual:\t", data.body(ee_id).xpos)
 
        # Send commands to the robot
        q = np.array([theta1, theta2, theta3, theta4])
        # q = np.array([0, 0, 0, 0])
        data.ctrl = q
        mujoco.mj_step(model, data)
        v.sync()
        time.sleep(update_freq)

In [24]:
## --- Jacobian IK Function --------------------------------------------------- ##
def ik_step_jacobian(model, data, goal_pos, damping=0.01, scale=1.0):
    """
    Compute one IK update step using Damped Least Squares (Jacobian pseudo-inverse),
    including orientation control so the end effector always points downward.
    """
    ee_id = model.body('ee').id

    mujoco.mj_forward(model, data)

    # --- Position error ---
    ee_pos = data.body(ee_id).xpos.copy()
    pos_error = goal_pos - ee_pos

    # --- Compute rotation matrix ---
    R = data.body(ee_id).xmat.reshape(3, 3)
    ee_z_axis = R[:, 2]                   # current EE Z-axis (world)
    world_down = np.array([0, 0, -1])     # desired direction

    # Orientation error via cross product (minimal change)
    orient_error = np.cross(ee_z_axis, world_down)

    # --- Compute Jacobians ---
    Jp = np.zeros((3, model.nv))
    Jr = np.zeros((3, model.nv))
    mujoco.mj_jac(model, data, Jp, Jr, ee_pos.copy(), ee_id)

    Jp = Jp[:, :4]
    Jr = Jr[:, :4]

    # --- Combine for full IK ---
    w_rot = 0.2   # rotation weight (tune 0.1–0.3)
    J_full = np.vstack([Jp, w_rot * Jr])
    err_full = np.hstack([pos_error, w_rot * orient_error])

    # Damped least squares
    JJt = J_full @ J_full.T
    dtheta = J_full.T @ np.linalg.inv(JJt + damping * np.eye(JJt.shape[0])) @ err_full

    return scale * dtheta


def instructions_to_coordinates(instructions, start=np.array([ball_pos_x, ball_pos_y, ball_pos_z]), step_size=cell_size):
    coordinates = [start]
    z_rise = 0.05  # keep arm raised
    coordinates[0][2] = ball_pos_z + z_rise
    for inst in instructions:
        x, y, z = coordinates[-1].copy()
        if inst == "U": 
            y += step_size
        elif inst == "D": 
            y -= step_size
        elif inst == "R": 
            x += step_size
        elif inst == "L": 
            x -= step_size
        else: 
            raise ValueError(f"Invalid instruction: {inst}")
        coordinates.append(np.array([x, y, ball_pos_z + z_rise]))
    return coordinates


def dist(point1, point2):
    return np.linalg.norm(np.array(point2) - np.array(point1))


## --- Main Simulation Loop --------------------------------------------------- ##

model = mujoco.MjModel.from_xml_string(robot_4dof)
data  = mujoco.MjData(model)

ee_id = model.body('ee').id
base_id = model.body('base').id
ball_id = model.body('ball').id

base_xpos = data.body(base_id).xpos.copy()

tol = 0.03
update_freq = 1/60

instructions = maze.start_end_to_instruction()
start_ball = np.array([ball_pos_x, ball_pos_y, ball_pos_z])
coordinates = instructions_to_coordinates(instructions, start=start_ball)
coord_index = 0

with mujoco.viewer.launch_passive(model, data) as v:
    while v.is_running():

        # ---- Compute target point ---- #
        x_goal, y_goal, z_goal = coordinates[coord_index]
        ee_pos = data.body(ee_id).xpos
        goal = np.array([x_goal, y_goal, z_goal])

        # Errors
        error = dist(ee_pos, goal)
        print("Error: ", error)
        # xy_error = np.linalg.norm(ee_pos[:2] - goal[:2])

        # advance to next coordinate only when fully reached
        if error < tol:
            coord_index += 1
            if coord_index >= len(coordinates):
                coord_index = len(coordinates) - 1

        # ---- IK Update ---- #
        dtheta = ik_step_jacobian(
            model,
            data,
            goal,
            damping=0.05,
            scale=0.5
        )

        theta_curr = data.qpos[:4].copy()
        theta_cmd = theta_curr + dtheta

        # keep final joint aligned for pointing down
        theta_cmd[3] = np.pi - (theta_cmd[1] + theta_cmd[2])

        data.ctrl[:] = theta_cmd

        print("Goal:\t", goal)
        print("Actual:\t", data.body(ee_id).xpos)

        mujoco.mj_step(model, data)
        v.sync()
        time.sleep(update_freq)


ValueError: XML Error: bad format in attribute 'face'
Element 'mesh', line 9
